In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/electricity-capability-nested/electricity_capability_nested.json
/kaggle/input/electricity-sales/electricity_sales.csv


In [2]:
import json

In [3]:
def extract_tabular_data(file_path: str):
    """Extract data from a tabular file_format, with pandas."""
    if "csv" in file_path:
        return pd.read_csv(file_path)
    elif "parquet" in file_path:
        return pd.read_parquet(file_path)
    else:
        raise Exception('Warning: Invalid file extension. Please try with .csv or .parquet!')

In [4]:
def extract_json_data(file_path):
    """Extract and flatten data from a JSON file."""
    if "json" in file_path:
        with open(file_path, "r") as json_file:
            raw_data = json.load(json_file)
        return pd.json_normalize(raw_data)
    else:
        print('enter a json file please')

In [5]:
def transform_electricity_sales_data(raw_data: pd.DataFrame):
    """
    Transform electricity sales to find the total amount of electricity sold
    in the residential and transportation sectors. """
    # drop NA records in price
    raw_data.dropna(subset=["price"], inplace=True)
    # keep records with sectorName of resid or transp
    cleaned_df = raw_data.loc[raw_data["sectorName"].isin(["residential", "transportation"]), :]
    # month column
    cleaned_df['year'] = cleaned_df['period'].str[0:4]
    # year column
    cleaned_df['month'] = cleaned_df['period'].str[5:]
    # transformed df
    cleaned_df = cleaned_df.loc[:, ["year", "month", "stateid", "price", "price-units"]]
    return cleaned_df

In [6]:
def load(dataframe: pd.DataFrame, file_path: str):
    """Load a DataFrame to a file in either CSV or Parquet format."""
    if file_path.endswith(".csv"):
        dataframe.to_csv(file_path)
    elif file_path.endswith(".parquet"):
        dataframe.to_parquet(file_path)
    else:
        raise Exception(f"Warning: {filepath} is not a valid file type. Please try again!")

In [7]:
raw_electricity_capability_df = extract_json_data("/kaggle/input/electricity-capability-nested/electricity_capability_nested.json")
raw_electricity_sales_df = extract_tabular_data("/kaggle/input/electricity-sales/electricity_sales.csv")

cleaned_electricity_sales_df = transform_electricity_sales_data(raw_electricity_sales_df)

load(raw_electricity_capability_df, "loaded_electricity_capability.parquet")
load(cleaned_electricity_sales_df, "loaded_electricity_sales.csv")

<ipython-input-5-d8014d6e0da0>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['year'] = cleaned_df['period'].str[0:4]
<ipython-input-5-d8014d6e0da0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['month'] = cleaned_df['period'].str[5:]
